In [ ]:
# Cell 1: Setup and Libraries
import requests
import json
import pandas as pd
from typing import List, Dict, Any, Optional
import yaml

# Dash imports
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
# import dash_bootstrap_components as dbc

# Constants for the API (replace with your actual credentials)
MIDAS_USERNAME = "your_username"  # Replace with your MIDAS Username
MIDAS_PASSWORD = "your_password"  # Replace with your MIDAS Password

# A sample RIN (Rate ID) to test the fetching function. 
# This should be a valid RIN from the catalog.
RIN_TO_FETCH = "CA00000000010100000001"

ImportError: cannot import name 'Lists' from 'typing' (c:\Users\bradw\miniconda3\envs\statistics-env\Lib\typing.py)

In [ ]:
with open("midas_config.yaml") as fp:
    config = yaml.safe_load(fp)
midas_config = config.get("midas", {})
MIDAS_USERNAME = midas_config.get("username")
MIDAS_PASSWORD = midas_config.get("password")
print(MIDAS_USERNAME)
print(MIDAS_PASSWORD)

In [ ]:
# Cell 2: API Functions and Data Tool Class

# --- Core API Functions ---

def GetToken() -> str:
    """Retrieves a Bearer Token for API authorization."""
    url = 'https://midasapi.energy.ca.gov/api/Token'
    headers = {'accept': 'application/json'}
    data = {
        'grant_type': 'password',
        'username': MIDAS_USERNAME,
        'password': MIDAS_PASSWORD
    }
    
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    token_data = response.json()
    
    # Check for 'access_token' in the response
    token = token_data.get('access_token')
    if not token:
        raise ValueError("Token endpoint did not return an access_token.")
        
    print("Token successfully retrieved.")
    return token

def GetCatalog(token: str) -> List[Dict[str, Any]]:
    """
    Retrieves the complete list of all active MIDAS rates (RINs).
    
    Note: The correct endpoint is ValueData?SignalType=0, not /api/catalogdata.
    """
    headers = {'accept': 'application/json', 'Authorization': "Bearer " + token}
    
    # ⚠️ CORRECTED ENDPOINT to fetch all RINs
    url = 'https://midasapi.energy.ca.gov/api/ValueData?SignalType=0'
    
    catalog_response = requests.get(url, headers=headers)
    catalog_response.raise_for_status() 
    
    return json.loads(catalog_response.text)

def GetValue(token: str, rin_id: str, start_date: str, end_date: str) -> pd.DataFrame:
    """Retrieves historical data for a specific RIN and date range."""
    headers = {'accept': 'application/json', 'Authorization': "Bearer " + token}
    
    # Endpoint for historical data
    url = f'https://midasapi.energy.ca.gov/api/HistoricalData/{rin_id}?startdate={start_date}&enddate={end_date}'
    
    data_response = requests.get(url, headers=headers)
    data_response.raise_for_status()
    
    # Parse the response and clean the data
    data = data_response.json()
    df = pd.DataFrame(data)
    
    # Data Cleaning/Preparation
    if 'Time' in df.columns and 'Value' in df.columns:
        df['Time'] = pd.to_datetime(df['Time'])
        df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
        df = df.dropna(subset=['Value'])
        return df.set_index('Time')
    else:
        print(f"Error: Missing 'Time' or 'Value' columns for RIN {rin_id}")
        return pd.DataFrame()


# --- MidasDataTool Class ---

class MidasDataTool:
    """A wrapper class to manage API token and data fetching."""
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.token = self._get_initial_token()
        self.catalog_rins = self.get_all_catalog_rins()

    def _get_initial_token(self) -> str:
        """Initial token retrieval, can be extended for refresh logic."""
        try:
            return GetToken()
        except Exception as e:
            print(f"Initial token retrieval failed: {e}")
            return ""

    def get_all_catalog_rins(self) -> Dict[str, str]:
        """Fetches the entire catalog and returns a dictionary of RateName: RIN ID."""
        if not self.token:
            return {}
        try:
            catalog_data = GetCatalog(self.token)
            rin_dict = {}
            for item in catalog_data: 
                rin_id = item.get('RateID')
                rin_name = item.get('Description')
                
                if rin_id and rin_name:
                    # Storing as {User-Friendly Name: RIN_ID}
                    rin_dict[rin_name] = rin_id 
                    
            if not rin_dict:
                 print("Warning: Catalog data retrieved, but no RINs were found.")
            return rin_dict

        except Exception as e:
            print(f"Error fetching MIDAS catalog: {e}")
            return {}
            
    def fetch_data(self, rin_id: str, start_date: str, end_date: str) -> pd.DataFrame:
        """Fetches and returns the DataFrame for a specific RIN and date range."""
        if not self.token:
            print("Error: No valid token available.")
            return pd.DataFrame()
        try:
            return GetValue(self.token, rin_id, start_date, end_date)
        except Exception as e:
            print(f"Error fetching data for RIN {rin_id}: {e}")
            return pd.DataFrame()

In [ ]:
# Cell 3: Data Initialization

# Initialize the data tool
midas_tool = MidasDataTool(MIDAS_USERNAME, MIDAS_PASSWORD)

# Get the list of RINs for the dropdown
rin_options = [{"label": name, "value": rin_id} for name, rin_id in midas_tool.catalog_rins.items()]

# Set default values for the initial chart
DEFAULT_RIN_ID = rin_options[0]['value'] if rin_options else RIN_TO_FETCH
DEFAULT_RIN_NAME = rin_options[0]['label'] if rin_options else "Default Rate"
DEFAULT_START_DATE = '2024-01-01'
DEFAULT_END_DATE = '2024-01-07'

# Fetch initial data for the default RIN
initial_df = midas_tool.fetch_data(
    rin_id=DEFAULT_RIN_ID,
    start_date=DEFAULT_START_DATE,
    end_date=DEFAULT_END_DATE
)

print(f"Tool initialized. Found {len(midas_tool.catalog_rins)} RINs. Initial data fetched.")

In [ ]:
# Cell 4: Dash App Layout

# Initialize the Dash app with a Bootstrap theme
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.H1("CEC MIDAS API Data Dashboard", className="my-4 text-primary"),

    # --- Controls Row ---
    dbc.Row([
        dbc.Col([
            html.Label("Select Rate (RIN):"),
            dcc.Dropdown(
                id='rin-dropdown',
                options=rin_options,
                value=DEFAULT_RIN_ID,
                searchable=True,
                clearable=False
            )
        ], md=6),

        dbc.Col([
            html.Label("Select Date Range:"),
            dcc.DatePickerRange(
                id='date-range-picker',
                start_date=DEFAULT_START_DATE,
                end_date=DEFAULT_END_DATE,
                display_format='YYYY-MM-DD',
                min_date_allowed='2020-01-01', # MIDAS data typically starts around this time
                max_date_allowed=pd.to_datetime('today').strftime('%Y-%m-%d'),
            )
        ], md=6),
    ], className="mb-4"),
    
    html.Hr(), # Horizontal separator

    # --- Output Graph and Table ---
    dbc.Row([
        dbc.Col(
            dcc.Graph(id='time-series-chart'), 
            width=12
        )
    ]),
    
    dbc.Row([
        dbc.Col(
            html.Div(id='data-summary'),
            width=12
        )
    ])

], fluid=True)

In [ ]:
# Cell 5: Dash App Callbacks and Run

@app.callback(
    Output('time-series-chart', 'figure'),
    Output('data-summary', 'children'),
    [
        Input('rin-dropdown', 'value'),
        Input('date-range-picker', 'start_date'),
        Input('date-range-picker', 'end_date')
    ]
)
def update_graph(selected_rin_id, start_date, end_date):
    """Callback to fetch new data and update the plot and summary table."""
    
    if not selected_rin_id or not start_date or not end_date:
        return px.scatter(), html.Div("Please select a RIN and a valid date range.")

    # Fetch new data
    df = midas_tool.fetch_data(selected_rin_id, start_date, end_date)
    
    # 1. Create Plotly Figure
    if not df.empty:
        # Get the friendly name for the title
        rin_name = next((item['label'] for item in rin_options if item['value'] == selected_rin_id), selected_rin_id)
        
        fig = px.line(
            df.reset_index(), # Reset index to use 'Time' as a column
            x='Time', 
            y='Value', 
            title=f'Price/Value Time Series for: {rin_name}',
            labels={'Time': 'Time (PST)', 'Value': 'Value (e.g., Price in $/kWh)'}
        )
        fig.update_layout(xaxis_title="Date and Time", yaxis_title="Value")
    else:
        fig = px.scatter(title="No Data Available for Selected Parameters")
        
    # 2. Create Summary Table
    if not df.empty:
        summary = html.Div([
            html.H4("Data Statistics"),
            dbc.Table.from_dataframe(
                df['Value'].describe().reset_index().rename(columns={'index': 'Statistic', 'Value': 'Value'}),
                striped=True,
                bordered=True,
                hover=True,
                className="table-sm"
            )
        ])
    else:
        summary = html.Div("No data summary available.")

    return fig, summary

# Run the app
if __name__ == '__main__':
    # Set to debug=True for local development
    # Change to debug=False for production
    print("\nStarting Dash App... navigate to http://127.0.0.1:8050/")
    app.run_server(debug=True)

In [ ]:
# Cell 6: Define Dash Callback (UPDATED)

if not combined_df.empty:
    @app.callback(
        Output('price-time-series-plot', 'figure'),
        [
            # Existing Inputs
            Input('rate-dropdown', 'value'),
            Input('date-range-picker', 'start_date'),
            Input('date-range-picker', 'end_date'),
            
            # --- NEW CHARACTERISTIC INPUTS ---
            Input('daytype-dropdown', 'value'),
            Input('ratetype-dropdown', 'value'),
            Input('state-dropdown', 'value'),
            # Add more inputs here if you added more dropdowns in Cell 5
        ]
    )
    def update_graph(selected_rates, start_date, end_date, selected_daytypes, selected_ratetypes, selected_states):
        df_filtered = combined_df.copy()
        df_tz = df_filtered.index.tz
        
        # --- Filtering by Characteristics (Sequential Filtering) ---
        
        # Helper for list filtering
        def apply_multi_filter(df, col, values):
            if values and isinstance(values, list) and values != [None]:
                return df[df[col].isin(values)]
            return df
        
        # Apply filters in order
        df_filtered = apply_multi_filter(df_filtered, 'Daytype', selected_daytypes)
        df_filtered = apply_multi_filter(df_filtered, 'RateType', selected_ratetypes)
        df_filtered = apply_multi_filter(df_filtered, 'State', selected_states)
        df_filtered = apply_multi_filter(df_filtered, 'rate_name', selected_rates) # Original Rate Name filter

        # --- Filtering by Timeframe (Timezone-Aware Fix) ---
        if start_date:
            start_ts_aware = pd.to_datetime(start_date).tz_localize(df_tz)
            df_filtered = df_filtered[df_filtered.index >= start_ts_aware]
        
        if end_date:
            end_ts_naive = pd.to_datetime(end_date) + pd.Timedelta(days=1)
            end_ts_aware = end_ts_naive.tz_localize(df_tz)
            df_filtered = df_filtered[df_filtered.index < end_ts_aware]

        # --- Empty Data Check and Plot Generation ---
        if df_filtered.empty:
            return {'data': [], 'layout': {'title': 'No Data Found for Selection', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}}}

        fig = px.line(
            df_filtered.reset_index(),
            x='start_time',
            y='price',
            color='rate_name', # Still color by rate_name to distinguish the lines
            title='Filtered Rate Time Series',
            labels={'start_time': f'Time ({df_tz.zone})', 'price': 'Price ($/MWh)', 'rate_name': 'Rate'}
        )
        fig.update_layout(transition_duration=500)
        
        return fig

In [ ]:
# Cell 7: Run the Dash App

if not combined_df.empty:
    # Use the current method for running the Dash application.
    # The 'debug=True' argument is still valid for enabling hot reloading.
    app.run(debug=True) 